# 🔎 Análisis Exploratorio: La Calidad del Aire en Madrid (2001-2022)

Este proyecto utiliza el dataset “MadridPolution2001-2022.csv”, que contiene registros horarios de contaminantes atmosféricos medidos en la estación Escuelas Aguirre (Madrid) desde enero de 2001 hasta marzo de 2022. El objetivo es analizar la calidad del aire en el centro de Madrid a lo largo de dos décadas y descubrir cómo ha cambiado la contaminación en la ciudad, identificando tendencias a largo plazo, patrones estacionales y episodios de polución extrema. Para ello, se estudiarán los registros horarios de 14 contaminantes clave, tomados desde enero de 2001 hasta marzo de 2022.

A continuación, se detalla qué es cada uno de estos contaminantes y por qué es importante medirlos.

---

## ¿Qué contaminantes se miden y qué significan?

Los datos recogidos incluyen diferentes tipos de partículas y gases que afectan tanto a nuestra salud como al medio ambiente. Se pueden agrupar en las siguientes categorías:

### 💨 Partículas en suspensión: El polvo que no se ve

Son pequeñas partículas sólidas y líquidas que flotan en el aire. Su peligrosidad depende de su tamaño: cuanto más pequeñas, más profundamente pueden penetrar en nuestros pulmones.

* **PM10**: Partículas con un diámetro inferior a 10 micrómetros. Incluyen polvo, polen, cenizas y hollín. Pueden irritar los ojos, la nariz y la garganta.
* **PM2.5**: Partículas con un diámetro inferior a 2.5 micrómetros, también conocidas como "partículas finas". Son mucho más peligrosas, ya que pueden llegar a los alvéolos pulmonares e incluso pasar al torrente sanguíneo. Se originan principalmente en la quema de combustibles de vehículos e industria.

### 🚗 Gases relacionados con el tráfico y la combustión

La mayoría de estos gases se producen al quemar combustibles como la gasolina, el diésel o el gas natural. Son los principales responsables de la "boina" de contaminación de las grandes ciudades.

* **CO (Monóxido de carbono)**: Un gas tóxico que se genera cuando la combustión es incompleta. Su principal fuente en las ciudades son los tubos de escape de los coches.
* **NO (Monóxido de nitrógeno)** y **NO2 (Dióxido de nitrógeno)**: Comúnmente agrupados como **NOx (Óxidos de nitrógeno)**, son gases que se forman a altas temperaturas en los motores de los vehículos y en las centrales eléctricas. El NO2 es el gas de color marrón-rojizo que se ve en las nubes de contaminación y está directamente relacionado con enfermedades respiratorias.
* **SO2 (Dióxido de azufre)**: Proviene principalmente de la quema de combustibles fósiles con contenido de azufre, como el carbón y el petróleo, en la industria y la calefacción. Es uno de los causantes de la lluvia ácida.

### ☀️ Ozono "malo": Un contaminante de verano

* **O3 (Ozono)**: A diferencia del ozono bueno de la estratosfera que nos protege del sol, el ozono a nivel del suelo es un contaminante. No se emite directamente, sino que se forma cuando los óxidos de nitrógeno (NOx) y otros compuestos reaccionan con la luz solar. Por eso, sus niveles suelen ser más altos en días soleados y calurosos, provocando irritación en las vías respiratorias.

### ⛽ Compuestos Orgánicos Volátiles (COVs)

Son un grupo amplio de sustancias químicas que se evaporan fácilmente a temperatura ambiente. Proceden de los combustibles, disolventes, pinturas y también de los tubos de escape.

* **BEN (Benceno)**, **TOL (Tolueno)** y **EBE (Etilbenceno)**: Son compuestos aromáticos derivados del petróleo, muy presentes en la gasolina. Son tóxicos y el benceno, en particular, es un conocido cancerígeno.
* **CH4 (Metano)**, **NMHC (Hidrocarburos no metánicos)** y **TCH (Hidrocarburos totales)**: Estos términos agrupan a todos los compuestos de hidrocarburos en el aire. El metano (CH4) es un potente gas de efecto invernadero, mientras que los NMHC son importantes porque contribuyen a la formación de ozono. El TCH es simplemente la suma de ambos.

El análisis de estas variables nos permitirá obtener una radiografía completa de la evolución del aire que respiramos en Madrid y entender mejor el impacto de nuestras actividades diarias en el entorno urbano.

## 🌱 Alba (limpieza y calidad de datos)

Convertimos la columna Time, originalmente en formato string, a tipo datetime para asegurar consistencia temporal y permitir análisis por fecha y hora. También creamos nuevas columnas (year, month, day, hour) a partir de la columna de fecha para facilitar el análisis exploratorio por periodos de tiempo.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# Cargar el dataset
csv_path = r"data/MadridPolution2001-2022.csv"

# Leer sin parsear fechas inicialmente para detectar el nombre de columna correcto
df = pd.read_csv(csv_path)

# --- LIMPIEZA DE FECHA ---
# La columna "time" la convertimos directamente a datetime
df["Time"] = pd.to_datetime(df["Time"], errors="coerce") # Con errors="coerce"

# Verificamos si se han generado nulos al convertir la columna "Time" a formato datetime
print("Nulos en la columna Time después de la conversión: ", df["Time"].isna().sum())

# Creamos columnas auxiliares para explorar temporalmente el dataset, pero de momento no haremos una limpieza estricta
df["year"] = df["Time"].dt.year
df["month"] = df["Time"].dt.month
df["day"] = df["Time"].dt.day
df["hour"] = df["Time"].dt.hour


# Mostramos los resultados
print("Columnas detectadas en el CSV:", df.columns.tolist())
print("Dimensiones del dataset:", df.shape)
print(df.head(10))

Analizamos la presencia de valores nulos en todas las columnas y calculamos su porcentaje respecto al total de filas, identificando aquellas variables con mayor pérdida de información

In [ ]:
# Contamos la cantidad de nulos que hay en cada columna
total_filas = len(df)

nulos_por_columna = df.isnull().sum()

conteo_de_nulos = nulos_por_columna.to_frame("Nulos").assign(Porcentaje=lambda x: (x["Nulos"] / total_filas * 100).round(2)).sort_values("Nulos", ascending=False)

print("--- Cantidad total de nulos en cada columna y porcentaje---")
print(conteo_de_nulos)

Añadimos un gráfico de barras para poder representar de forma más visual cuales son las columnas que tienen una mayor cantidad de nulos en sus filas. De esta manera podemos determinar qué variables presentan problemas de calidad de datos debido a la alta proporción de valores ausentes, lo que afecta la representatividad de las mediciones y debe considerarse en futuros análisis.

In [ ]:
conteo_de_nulos['Porcentaje'].plot(kind="bar", color='red', legend=False)
plt.title("Porcentaje de valores nulos en cada columna")
plt.xlabel("Columnas del DataSet")
plt.ylabel("Valores nulos (%)")

plt.grid(True, axis='y', linestyle='--', alpha=0.7)

plt.ylim(0, 100)
ejes = plt.gca()
ejes.yaxis.set_major_formatter(mticker.FuncFormatter(lambda y, _: f'{int(y)}%'))
plt.show()

En base a los resultados, podemos comprobar que la columna que tiene un mayor valor de nulos es la de `PM25`, con casi un 40% de valores nulos. Después, tenemos otras tres columnas con un porcentaje de valores nulos por debajo del 20% que serían `NMHC`, `TCH` y `CH4`. Después, el resto de columnas tienen porcentajes de nulos muy bajos y en caso de las columnas que representan la fecha (`Time`, `year`, `month`, `day`, `hour`) tienen un porcentaje del 0%, por lo que a pesar de que haya contaminantes con un porcentaje de nulos más o menos alto, podremos realizar un análasis temporal completo, ya que tenemos todos los valores con fecha y hora.

Comprobamos la existencia de filas duplicadas en el dataset y evaluamos su impacto, eliminándolas en caso de ser necesario para evitar sesgos en el análisis.

In [ ]:
duplicados = df.duplicated().sum()
print(f"Numero de filas duplicadas: {duplicados}")

# En caso de que detectase algún duplicado, con el método drop_duplicates eliminaría la columna entera
if duplicados > 0:
    df.drop_duplicates()
    print("Duplicados eliminados. Nuevo tamaño del dataset: ", df.shape)

Revisamos los tipos de datos de cada columna para asegurar que las variables numéricas se encuentren en formato correcto (float/int) y no en formato texto, evitando errores en el procesamiento posterior.

In [10]:
print("\nTipos de datos detectados en el dataset:")
print(df.dtypes)

# Comprobamos si hay columnas numéricas cargadas como texto
numeric_cols = df.select_dtypes(include=["object"]).columns
print("\nColumnas tipo 'object' (texto):", numeric_cols.tolist())


Tipos de datos detectados en el dataset:
Time     datetime64[ns, UTC]
BEN                  float64
CH4                  float64
CO                   float64
EBE                  float64
NMHC                 float64
NO                   float64
NO2                  float64
NOx                  float64
O3                   float64
PM10                 float64
PM25                 float64
SO2                  float64
TCH                  float64
TOL                  float64
year                   int32
month                  int32
day                    int32
hour                   int32
dtype: object

Columnas tipo 'object' (texto): []


Analizamos si existen valores anómalos o fuera de rango en las variables (ej. concentraciones negativas o valores extremadamente altos) para garantizar la coherencia de las mediciones.

In [11]:
# Buscar valores negativos en las columnas numéricas
negativos = (df.select_dtypes(include=["float64", "int64"]) < 0).sum()
print("\nValores negativos por columna:")
print(negativos[negativos > 0])

# Ver rango de cada variable (mínimo y máximo)
rangos = df.describe().T[["min", "max"]]
print("\nRangos de valores por columna:")
print(rangos)


Valores negativos por columna:
Series([], dtype: int64)

Rangos de valores por columna:
          min     max
BEN       0.0    43.0
CH4       0.0     4.0
CO        0.0    10.0
EBE       0.0    81.0
NMHC      0.0     9.0
NO        1.0  1041.0
NO2       4.0   402.0
NOx       5.0  1910.0
O3        0.0   199.0
PM10      1.0   367.0
PM25      0.0   215.0
SO2       1.0   158.0
TCH       0.0    10.0
TOL       0.0   174.0
year   2001.0  2022.0
month     1.0    12.0
day       1.0    31.0
hour      0.0    23.0


## 📊 Robert (análisis general y descriptivo)

- Promedio anual: ¿Cuál es la concentración media de cada contaminante por año? (tabla resumen por año y contaminante).
- Mejora: Visualizar la evolución anual de los principales contaminantes con gráficos de líneas.


- Mes con peor calidad del aire: ¿Qué mes tuvo la media más alta de NO₂ cada año? (gráfico de barras mensual por año).
- Mejora: Mostrar el mes más crítico para cada año y visualizar la tendencia mensual agregada.

- Distribución de O₃: ¿Cuál es el rango de concentraciones de O₃ más frecuente? (histograma global y por año).
- Mejora: Analizar la variabilidad anual y estacional de O₃.

- Picos diarios: ¿Qué día presentó la concentración más alta de PM10 en cada año?
- Mejora: Visualizar los días de picos extremos y analizar si hay patrones estacionales o de eventos puntuales.

## 🌍 David (comparaciones y correlaciones)

- Estacionalidad en un año: ¿Qué contaminante presenta mayores diferencias entre invierno y verano?
- Mejora: Analizar la estacionalidad de todos los contaminantes y visualizar la diferencia entre estaciones.

- Correlación: ¿Existe correlación entre los niveles diarios de NO₂ y CO en el periodo analizado?
- Mejora: Analizar correlaciones entre más contaminantes y visualizar la matriz de correlación.

- Laborables vs fines de semana: ¿Hay diferencias en los niveles de NO₂ promedio entre días laborables y fines de semana?
- Mejora: Analizar diferencias para más contaminantes y visualizar la variación semanal.

- Variación horaria: ¿A qué horas del día se concentran los picos de NO₂ en promedio durante el año? (curva horaria).
- Mejora: Analizar la variación horaria de más contaminantes y comparar entre años.